In [ ]:
from collections import OrderedDict
from datetime import datetime
import logging
import os
from pprint import pprint

import geojson
import geopandas as gpd
import fiona
import matplotlib as plt
from shapely.geometry import (
    LineString, mapping, shape
)
from shapely.ops import (
    split as shapely_split, transform
)

from python_gis.poc.landgrid_processor import LandgridProcessor
from python_gis.poc.io.pgsql import PgWriter
from python_gis.poc.tools.spatial import (
    remove_holes
)
import python_gis.poc.util.log_config

logger = logging.getLogger(__name__)

In [ ]:
!ls $DIML_HOME

In [ ]:
# Postgis
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

# WGS84 (epsg:4326)
gdb_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')
ddl_path = os.path.join(os.environ["DIML_HOME"], 'database', 'schema.sql')
out_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out')
idx_path = os.path.join(os.environ["DIML_HOME"], 'database', "indexes.sql")
test_path = os.path.join(os.environ["DATA_DIR"], 'mssql_test')

## Anti-Meridian Fix
- Splits polygon at the meridian and keeps the right half

In [ ]:
def shift_west(x, y, z=None):
    return tuple(filter(None, ((x - 360)%-360, y, z)))  # shift to west hemi

def unshift_west(x, y, z=None):
    return tuple(filter(None, ((x + 360), y, z)))  # shift back to east hemi

# def shift_east(x, y, z=None):
#     return tuple(filter(None, ((x + 360)%360, y, z))) # shift to east hemi

# def unshift_east(x, y, z=None):
#     return tuple(filter(None, ((x - 360), y, z))) # shift back to west hemi

# TODO: Try 150
def is_near_idl(x):
    return x < -150 or x > 150

In [ ]:
%%time

# WGS84 (epsg:4326)
gdb_src_path = os.path.join(gdb_path)
shp_out_path = os.path.join(out_path, 'test_counties_bbox_pg_v4.shp')  # test_states_bbox_pg_v4.shp, test_counties_bbox_pg.shp

west_meridian = LineString([(-180, 90), (-180, -90)])

with fiona.open(gdb_src_path, layer='Counties_US_WGS84') as src:  # Counties_US_WGS84, States_US
    print(f"Driver: {src.driver}, CRS: {src.crs}")
    
    # Copy the source schema and add a property.
    schema = src.schema.copy()
    schema['properties']['geobounds'] = 'str:254'  # new
    schema['geometry'] = 'Polygon'  # update
    
    with fiona.open(shp_out_path, 'w', driver='ESRI Shapefile', 
                    schema=schema, crs=src.crs) as tgt:
        
        for f in src:
            
#             pprint(f"Props: {f['properties']}")
            poly = shape(f['geometry'])
            min_x, _, max_x, _ = poly.bounds
            
            if not poly.is_valid:
                clean_poly = poly.buffer(0.0)
                assert clean_poly.is_valid
                assert clean_poly.geom_type in ('Polygon', 'MultiPolygon')
                poly = clean_poly
            
            # Check if near dateline.
            if is_near_idl(min_x) or is_near_idl(max_x):
                
                print(f"{f['properties']} near IDL.")
                pprint(f"Shape Envelope: {poly.bounds}")
                
                # Shift to west hemisphere
                shape_shift = transform(shift_west, poly)
                shape_envelope = shape_shift.envelope
                pprint(f"Shape Envelope after shift: {shape_shift.bounds}")

                if shape_envelope.intersects(west_meridian):
                    print(f"{f['properties']} crosses IDL.")
                    
                    _, right_half = shapely_split(shape_envelope, west_meridian)
                    
                    pprint(f"pre-half_{i}: {right_half.bounds}")
                    min_x, _, max_x, _ = right_half.envelope.bounds

                    # Shift back to the east hemisphere.
                    if min_x < -180 or max_x < -180:  # check west hemi
                        right_half = transform(unshift_west, right_half.envelope)

                    pprint(f"post-half_{i}: {right_half.bounds}")
                    pprint(f"post-half_{i} area: {right_half.area}")

                    # Keep larger area
                    f['geometry'] = mapping(right_half)
                    f['properties'].update(geobounds=geojson.dumps(f['geometry']))
                    tgt.write(f)


In [ ]:
%%time

# WGS84 (epsg:4326)
src_path = os.path.join(test_path, 'landgrid.County.shp')  # landgrid.State.shp, landgrid.County.shp
shp_out_path = os.path.join(out_path, 'landgrid.County.fme.shp')

with fiona.open(src_path) as src:
    print(f"Driver: {src.driver}, CRS: {src.crs}")
    
    # Copy the source schema and add a property.
    schema = src.schema.copy()
    schema['geometry'] = 'Polygon'  # update
    pprint(f"Schema: {schema}")
    
    with fiona.open(shp_out_path, 'w', driver='ESRI Shapefile', 
                    schema=schema, crs=src.crs) as tgt:
        
        for f in src:
            
            print(f"State: {f['properties']['StateName']}")
            pprint(f"BoundsGeoS: {f['properties']['BoundsGeoS']}")
            
            g_json = geojson.loads(f['properties']['BoundsGeoS'])
            state = shape(g_json)
            
            # Writes as python-geo-interface (GeoJSON-like)
            f['geometry'] = mapping(state)
            
            pprint(f"Geometry: {f['geometry']}")
            pprint(f"GeoJSON: {geojson.dumps(f['geometry'])}")

            tgt.write(f)


In [ ]:
g_json2 = '{"type":"Polygon","coordinates":[[[172.44245193800009,51.214336780000053],[-129.99567699399995,51.214336780000053],[-129.99567699399995,71.390079301000071],[172.44245193800009,71.390079301000071],[172.44245193800009,51.214336780000053]]]}'
bbox2 = geojson.loads(g_json2)

poly2 = shape(bbox2)

poly2os.path.join(os.environ["DATA_DIR"], 'shapefile_out')